In [1]:
import pandas as pd
import numpy as np
import torch

In [2]:
normal = pd.read_csv("C:/Users/tlseh/Desktop/WADI_14days_new.csv")

In [3]:
normal_drop = normal.drop(['Row','Date','Time'],axis='columns')

In [4]:
normal_drop_1 = normal_drop[0:1000]

In [5]:
normal_drop_1 = normal_drop_1.dropna(axis=1) # 값이 없는 열 제거 여기서는 행 4개가 제거되었다.

In [6]:
normal_drop_1

,1_AIT_001_PV,1_AIT_002_PV,1_AIT_003_PV,1_AIT_004_PV,1_AIT_005_PV,1_FIT_001_PV,1_LS_001_AL,1_LS_002_AL,1_LT_001_PV,1_MV_001_STATUS,...,3_MV_001_STATUS,3_MV_002_STATUS,3_MV_003_STATUS,3_P_001_STATUS,3_P_002_STATUS,3_P_003_STATUS,3_P_004_STATUS,LEAK_DIFF_PRESSURE,PLANT_START_STOP_LOG,TOTAL_CONS_REQUIRED_FLOW
0,171.155,0.619473,11.5759,504.645,0.318319,0.001157,0,0,47.8911,1,...,1,1,1,1,1,1,1,67.9651,1,0.68
1,171.155,0.619473,11.5759,504.645,0.318319,0.001157,0,0,47.8911,1,...,1,1,1,1,1,1,1,67.9651,1,0.68
2,171.155,0.619473,11.5759,504.645,0.318319,0.001157,0,0,47.8911,1,...,1,1,1,1,1,1,1,67.9651,1,0.68
3,171.155,0.607477,11.5725,504.673,0.318438,0.001207,0,0,47.7503,1,...,1,1,1,1,1,1,1,67.1948,1,0.68
4,171.155,0.607477,11.5725,504.673,0.318438,0.001207,0,0,47.7503,1,...,1,1,1,1,1,1,1,67.1948,1,0.68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,174.938,0.559481,11.7524,484.411,0.286654,1.907730,0,0,40.5656,2,...,1,1,1,1,1,1,1,63.8467,1,0.68
996,174.938,0.559481,11.7524,484.411,0.286654,1.907730,0,0,40.5656,2,...,1,1,1,1,1,1,1,63.8467,1,0.68
997,174.938,0.559481,11.7524,484.411,0.286654,1.907730,0,0,40.5656,2,...,1,1,1,1,1,1,1,63.8467,1,0.68
998,174.213,0.541483,11.7854,482.401,0.286422,2.003600,0,0,40.8926,2,...,1,1,1,1,1,1,1,63.9204,1,0.68


In [7]:
normal_drop_1.isnull().sum().sum() # 성분에서 NaN이 있으면 1이상의 값이 나오게 된다. pd.DataFrame.isnull().sum().sum()

0

In [8]:
from sklearn import preprocessing       # 정규화를 반드시 진행해야 하는 건지?
min_max_scaler = preprocessing.MinMaxScaler()

x = normal_drop_1.values
x_scaled = min_max_scaler.fit_transform(x)
normal_drop_1_scaled = pd.DataFrame(x_scaled)

print(normal_drop_1_scaled)

          0         1         2         3         4         5    6    7    \
0    0.006593  0.529401  0.252453  0.919934  0.824437  0.000094  0.0  0.0   
1    0.006593  0.529401  0.252453  0.919934  0.824437  0.000094  0.0  0.0   
2    0.006593  0.529401  0.252453  0.919934  0.824437  0.000094  0.0  0.0   
3    0.006593  0.470590  0.240948  0.921092  0.827501  0.000118  0.0  0.0   
4    0.006593  0.470590  0.240948  0.921092  0.827501  0.000118  0.0  0.0   
..        ...       ...       ...       ...       ...       ...  ...  ...   
995  0.662908  0.235287  0.849746  0.083127  0.009064  0.923259  0.0  0.0   
996  0.662908  0.235287  0.849746  0.083127  0.009064  0.923259  0.0  0.0   
997  0.662908  0.235287  0.849746  0.083127  0.009064  0.923259  0.0  0.0   
998  0.537127  0.147052  0.961421  0.000000  0.003090  0.969679  0.0  0.0   
999  0.537127  0.147052  0.961421  0.000000  0.003090  0.969679  0.0  0.0   

          8    9    ...  113  114  115  116  117  118  119       120  121  

In [9]:
normal_drop_1 = np.array(normal_drop_1) # Dataframe을 numpy로 변환

In [10]:
normal_drop_1.shape
print("before transpose :{}".format(normal_drop_1.shape)) # 데이터셋 입력전 rnn 모양을 맞춰주기 위해서 행열 transpose
normal_drop_1 = np.transpose(normal_drop_1)
print("after transpose :{}".format(normal_drop_1.shape))

before transpose :(1000, 123)
after transpose :(123, 1000)


In [11]:
#normal_drop_1[:,1:10].shape
len(normal_drop_1[0])

1000

In [12]:
from torch.utils.data import Dataset

class MyDataset(Dataset):
    def __init__(self, data, window, stride):
        self.data = data
        self.window = window
        self.stride = stride
        self.batch_size = (len(self.data[0]) - self.window) // self.stride

    def __getitem__(self, index):
        # print(index) index 확인할때 출력 해보려고 써놓음
        x = torch.tensor([])
        stack = torch.tensor(self.data[:,index + self.stride*index : index + self.window + self.stride*index]).unsqueeze(0)
        x = torch.cat((x,stack),dim=0)
        return x

    def __len__(self):
        return self.batch_size

In [13]:
normal_drop_1[:,0:5].shape

(123, 5)

In [14]:
Dataset = MyDataset(normal_drop_1,1,3)

In [15]:
len(Dataset)

333

In [20]:
Dataset.x

AttributeError: 'MyDataset' object has no attribute 'x'

In [16]:
xxx = 0
#for a in Dataset:
    #print(type(a))
    #print(a.shape)
    #print(a)
    #xxx +=1
   # print(xxx)

In [17]:
(len(Dataset) - Dataset.window)  // Dataset.stride

110

In [18]:
from torch.utils.data import DataLoader

BATCH_SIZE = ((len(Dataset) - Dataset.window) // Dataset.stride)

dataloader = DataLoader(Dataset,batch_size=BATCH_SIZE, shuffle=False)

In [19]:
x1 = 0
for batch_index, sample in enumerate(dataloader): # 데이터셋 만들때 (,) 행렬 제일 마지막 성분이 길이가 되도록 transpose 시켜주어야 할듯!
    print("Sample size is {}, Index is {}".format(sample.size(), batch_index))
    x1 += 1
    if x1 ==1 :
        break

Sample size is torch.Size([110, 1, 123, 1]), Index is 0
